### Settings and imports

In [1]:
%matplotlib notebook

In [2]:
import masserstein

In [3]:
import os

from masserstein import Spectrum, NMRSpectrum
from masserstein import estimate_proportions

from matplotlib import pyplot as plt

import numpy as np

import pulp

import pickle

import seaborn as sns
from textwrap import wrap

### Ground truth

In [4]:
protons = [10, 10, 8]

In [5]:
#from 0 to 3!!! (Not from 1 to 4)
variant = 0

In [6]:
molar_proportions = [
                    [0.3401, 0.3299, 0.3299],
                    [0.2525, 0.2475, 0.5],
                    [0.2538, 0.4949, 0.2513],
                    [0.2075, 0.3942, 0.3983]
                    ]

In [7]:
real_visible_proportions = [prop*prot for prop, prot in zip(molar_proportions[variant], protons)]
real_visible_proportions = [p/sum(real_visible_proportions) for p in real_visible_proportions]
real_visible_proportions

[0.22546015602929353, 0.428319968707217, 0.34621987526348963]

### Loading the data

In [8]:
official_names = ['Leucine', 'Isoleucine', 'Valine']

In [9]:
mix_files = ['mixture_I.csv', 'mixture_II.csv', 'mixture_III.csv', 'mixture_IV.csv']

In [10]:
mix = np.loadtxt(os.getcwd()+'/'+mix_files[variant], skiprows=0, usecols=range(2), 
                      delimiter=',',dtype=float)
ppm = mix[:,0]
mix = mix[:,1]

comp = np.loadtxt(os.getcwd()+'/library_BCAA_aligned.csv', skiprows=1, usecols=range(4),
                      delimiter=',', dtype=float)
ppm_components = comp[:,0]
components = comp[:,1:]

In [11]:
components_ints = []
names = []
for i in range(components.shape[1]):
    components_ints.append(components[:,i])
    names.append('comp'+str(i))

In [12]:
spectra = []
for i, comp_ints in enumerate(components_ints):
    spectra.append(NMRSpectrum(confs=list(zip(ppm_components, comp_ints)), protons=protons[i]))

In [13]:
mix = NMRSpectrum(confs=list(zip(ppm,mix)))

In [14]:
# mix.plot(profile=True)
# spectra[0].plot(profile=True)
# spectra[1].plot(profile=True)
# spectra[2].plot(profile=True)

### Alignment

In [15]:
def shift_one_spectrum(sp, how_much):
    res = sp
    old_confs = np.array(sp.confs)

    new_ppm_non_unique = old_confs[:,0] + how_much
    resampled = sp.resample(np.unique(new_ppm_non_unique) + how_much)
    new_ppm_unique = np.array(resampled.confs)[:,0]
    new_ints = np.array(resampled.confs)[:,1]
    res = NMRSpectrum(confs=list(zip(new_ppm_unique + how_much, new_ints)), protons=sp.protons)
    return(res)

### Preprocessing

In [16]:
def shift(list_of_spectra, epsilon=0.001):
    res = list_of_spectra
    shifted_unique = []
    old_confs = [np.array(sp.confs) for sp in list_of_spectra]
    min_of_min = min([min(old_conf[:,0]) for old_conf in old_confs])
    if min_of_min<0:
        return (min_of_min - epsilon)*(-1)
    else:
        return(0)

In [17]:
def make_nonnegative(spectrum):
    confs = np.array(spectrum.confs)
    neg = confs[:,1]<0
    confs[:,1][neg]=0
    new_confs = list(zip(confs[:,0],confs[:,1]))
    result = NMRSpectrum(confs=new_confs)
    return(result)

In [18]:
def common_mass_axis(list_of_spectra):
    all_mz = set()
    for sp in list_of_spectra:
        mz_set = set(np.array(sp.confs, np.dtype('float'))[:,0])
        all_mz = all_mz.union(mz_set)
    all_mz = np.array(sorted(all_mz))
    return all_mz

In [19]:
spectra_and_mixture = spectra + [mix]

In [20]:
shift_coef = shift(spectra_and_mixture)

In [21]:
preprocessed_spectra = []
for sp in spectra_and_mixture:
    sp2 = make_nonnegative(sp)
    sp2.sort_confs()
    sp2.merge_confs()
    sp2.normalize()
    preprocessed_spectra.append(sp2)

In [22]:
spectra_and_mixture = preprocessed_spectra
spectra = spectra_and_mixture[:-1]
mix = preprocessed_spectra[-1]
del(preprocessed_spectra)

In [23]:
cma = common_mass_axis(spectra_and_mixture)

In [24]:
preprocessed_spectra = []
for sp in spectra_and_mixture:
    preprocessed_spectra.append(sp.resample(cma))

In [25]:
spectra_and_mixture = preprocessed_spectra
del(preprocessed_spectra)

In [26]:
preprocessed_spectra = []
for sp in spectra_and_mixture:
    sp = make_nonnegative(sp)
    sp.sort_confs()
    sp.merge_confs()
    sp.normalize()
    preprocessed_spectra.append(sp)

In [27]:
spectra_and_mixture = preprocessed_spectra
spectra = spectra_and_mixture[:-1]
mix = preprocessed_spectra[-1]
del(preprocessed_spectra)

In [28]:
# mix.plot(profile=True)
# spectra[0].plot(profile=True)
# spectra[1].plot(profile=True)
# spectra[2].plot(profile=True)

### Removing unnecessary data points

In [29]:
def cut_spectra_to_region(list_of_spectra, lower_bound, upper_bound):
    res = []
    for sp in list_of_spectra:
        ppm = np.array(sp.confs)[:,0]
        ints = np.array(sp.confs)[:,1]
        ppm_in_region = np.logical_and(ppm>lower_bound, ppm<upper_bound)
        new_ppm = ppm[ppm_in_region]
        new_ints = ints[ppm_in_region]
        res.append(masserstein.NMRSpectrum(confs=list(zip(new_ppm, new_ints))))
    return res

In [30]:
#0.1, 4 without solvent peak
#0.1, 5.4
preprocessed_spectra = cut_spectra_to_region(spectra_and_mixture, 0.1, 4)

In [31]:
spectra_and_mixture = preprocessed_spectra
spectra = spectra_and_mixture[:-1]
mix = preprocessed_spectra[-1]
del(preprocessed_spectra)

In [32]:
preprocessed_spectra = []
for sp in spectra_and_mixture:
    sp2 = make_nonnegative(sp)
    sp2.sort_confs()
    sp2.merge_confs()
    sp2.normalize()
    preprocessed_spectra.append(sp2)

In [33]:
spectra_and_mixture = preprocessed_spectra
spectra = spectra_and_mixture[:-1]
mix = preprocessed_spectra[-1]
del(preprocessed_spectra)

In [34]:
for i, sp in enumerate(spectra):
    sp.protons = protons[i]

In [35]:
for i, name in enumerate(names):
    globals()[name] = spectra[i]

In [36]:
components_ints = []
for spectrum in spectra:
    components_ints.append(np.array(spectrum.confs)[:,1])#.reshape(-1,1))

In [37]:
ppm = np.array(mix.confs)[:,0]
mix_ints = np.array(mix.confs)[:,1]

In [38]:
official_names

['Leucine', 'Isoleucine', 'Valine']

In [39]:
labels = official_names + ['Mixture']
for i, sp in enumerate(spectra_and_mixture):
    sp.label = labels[i]

In [40]:
# fig, ax = plt.subplots()
# fig.set_size_inches(9, 4, forward=True)
# mix.plot(profile=True, color='black')
# comp0.plot(profile=True)
# comp1.plot(profile=True) 
# comp2.plot(profile=True) 
# ax.legend()

### Finding best kappa

In [41]:
import time
import pandas as pd
import pulp

In [42]:
lower_limit = 0.01
upper_limit = 0.31
step = 0.01

In [47]:
# list_of_estimation_results = []
# for kappa in np.arange(start=lower_limit, stop=upper_limit, step=step):
#     fixed_kappa = []
#     for kappa_prime in np.arange(start=lower_limit, stop=upper_limit, step=step):
#         start = time.time()
#         estimation = estimate_proportions(mix, spectra, 
#                                    MTD=kappa, MTD_th=kappa_prime, verbose=False, solver=pulp.GUROBI(msg=False),
#                                          what_to_compare='area')
#         end = time.time()
#         fixed_kappa.append(estimation['proportions'])
#         print('Estimation for '+str(kappa)+' and '+str(kappa_prime)+' done')
#         print('It took: '+str(end-start)+' seconds.')
#         print(estimation['proportions'])
#     list_of_estimation_results.append(fixed_kappa)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-11-10


/home/basia/.local/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Estimation for 0.01 and 0.01 done
It took: 2.9432554244995117 seconds.
[0.2392777852240008, 0.3994332683050013, 0.31778488053600107]
Estimation for 0.01 and 0.02 done
It took: 3.2169556617736816 seconds.
[0.2390894996000008, 0.4107516789170014, 0.294714814964001]
Estimation for 0.01 and 0.03 done
It took: 3.6210274696350098 seconds.
[0.23699148503800077, 0.41369062735900136, 0.29041503518400097]
Estimation for 0.01 and 0.04 done
It took: 3.5748767852783203 seconds.
[0.23459241278400078, 0.4164725186680014, 0.28547317473400097]
Estimation for 0.01 and 0.05 done
It took: 3.5762383937835693 seconds.
[0.23273058602300079, 0.41788575779000137, 0.28346252017600093]
Estimation for 0.01 and 0.060000000000000005 done
It took: 3.6168673038482666 seconds.
[0.23180526449400077, 0.4181888036440014, 0.28348985723900094]
Estimation for 0.01 and 0.06999999999999999 done
It took: 3.4565348625183105 seconds.
[0.23089529965500077, 0.4183830612600014, 0.28378488397500096]
Estimation for 0.01 and 0.08 done

Estimation for 0.03 and 0.02 done
It took: 3.3614022731781006 seconds.
[0.23633358628200077, 0.40392324277600133, 0.3357108367610011]
Estimation for 0.03 and 0.03 done
It took: 3.0758883953094482 seconds.
[0.23640669615100077, 0.41647171909700137, 0.31488252741300105]
Estimation for 0.03 and 0.04 done
It took: 3.2626686096191406 seconds.
[0.2325740002770008, 0.42352654001900136, 0.31045133938800107]
Estimation for 0.03 and 0.05 done
It took: 3.2938005924224854 seconds.
[0.2324478808500008, 0.42649826331800145, 0.304503681555001]
Estimation for 0.03 and 0.060000000000000005 done
It took: 3.534501552581787 seconds.
[0.23276894459000078, 0.42619060582700147, 0.301610942875001]
Estimation for 0.03 and 0.06999999999999999 done
It took: 3.777878522872925 seconds.
[0.23214555634300077, 0.42776084559100147, 0.299617852602001]
Estimation for 0.03 and 0.08 done
It took: 3.622175931930542 seconds.
[0.23192196581100077, 0.42810147747500144, 0.299222459761001]
Estimation for 0.03 and 0.09 done
It t

Estimation for 0.05 and 0.03 done
It took: 3.4743263721466064 seconds.
[0.23343354690100077, 0.40608603647000135, 0.34197352179400115]
Estimation for 0.05 and 0.04 done
It took: 3.448300838470459 seconds.
[0.23300540544400078, 0.4182821764220014, 0.32353976634600107]
Estimation for 0.05 and 0.05 done
It took: 3.4978532791137695 seconds.
[0.23313994168100077, 0.42384094970700137, 0.31324195982100106]
Estimation for 0.05 and 0.060000000000000005 done
It took: 3.9061295986175537 seconds.
[0.23270976813400077, 0.42555714296600144, 0.31104904203500106]
Estimation for 0.05 and 0.06999999999999999 done
It took: 3.7508249282836914 seconds.
[0.2325153486210008, 0.42664713370800145, 0.30880257143500106]
Estimation for 0.05 and 0.08 done
It took: 3.30019474029541 seconds.
[0.23256678669900077, 0.42892894854800145, 0.303778173813001]
Estimation for 0.05 and 0.09 done
It took: 3.3732223510742188 seconds.
[0.2325916183370008, 0.4300207046120014, 0.300630671740001]
Estimation for 0.05 and 0.099999999

...

Estimation for 0.22 and 0.18000000000000002 done
It took: 4.717931509017944 seconds.
[0.2521740237670008, 0.39326837402900133, 0.3385945752440011]
Estimation for 0.22 and 0.19 done
It took: 4.412408113479614 seconds.
[0.2574571696780008, 0.38523730883900126, 0.3406949818870011]
Estimation for 0.22 and 0.2 done
It took: 4.633276700973511 seconds.
[0.2592323420760009, 0.3823302213600013, 0.3416385609800011]
Estimation for 0.22 and 0.21000000000000002 done
It took: 4.564711093902588 seconds.
[0.2592323420760009, 0.3823302213600013, 0.3416385609800011]
Estimation for 0.22 and 0.22 done
It took: 4.758959531784058 seconds.
[0.2592323420760009, 0.3823302213600013, 0.3416385609800011]
Estimation for 0.22 and 0.23 done
It took: 4.59641170501709 seconds.
[0.2592323420760009, 0.3823302213600013, 0.3416385609800011]
Estimation for 0.22 and 0.24000000000000002 done
It took: 4.460497140884399 seconds.
[0.2592323420760009, 0.3823302213600013, 0.3416385609800011]
Estimation for 0.22 and 0.25 done
It t

Estimation for 0.24000000000000002 and 0.17 done
It took: 4.7655885219573975 seconds.
[0.2509725969010008, 0.39444031750100134, 0.3388914831750011]
Estimation for 0.24000000000000002 and 0.18000000000000002 done
It took: 4.684147119522095 seconds.
[0.2544016913310008, 0.3873447002530013, 0.34266295866200114]
Estimation for 0.24000000000000002 and 0.19 done
It took: 4.7602903842926025 seconds.
[0.26126224047200086, 0.3740607812620012, 0.34916856798400114]
Estimation for 0.24000000000000002 and 0.2 done
It took: 4.867054462432861 seconds.
[0.2676751751750009, 0.3614566115210012, 0.3554381205170012]
Estimation for 0.24000000000000002 and 0.21000000000000002 done
It took: 4.529264450073242 seconds.
[0.2676751751750009, 0.3614566115210012, 0.3554381205170012]
Estimation for 0.24000000000000002 and 0.22 done
It took: 4.585029125213623 seconds.
[0.2676751751750009, 0.3614566115210012, 0.3554381205170012]
Estimation for 0.24000000000000002 and 0.23 done
It took: 4.504698753356934 seconds.
[0.2

Estimation for 0.26 and 0.16 done
It took: 4.938391447067261 seconds.
[0.24663563970100083, 0.39926155461300133, 0.3388627558290011]
Estimation for 0.26 and 0.17 done
It took: 4.887355089187622 seconds.
[0.2522282442090008, 0.39011605668800126, 0.34231334883400116]
Estimation for 0.26 and 0.18000000000000002 done
It took: 4.871703863143921 seconds.
[0.2561710917840008, 0.3821277448340013, 0.3464504153610012]
Estimation for 0.26 and 0.19 done
It took: 4.856106519699097 seconds.
[0.2631433262970009, 0.36816264225400125, 0.35359675242300115]
Estimation for 0.26 and 0.2 done
It took: 4.917418479919434 seconds.
[0.2685385781320009, 0.35777029564400115, 0.35866705527300125]
Estimation for 0.26 and 0.21000000000000002 done
It took: 4.903806924819946 seconds.
[0.2685385781320009, 0.35777029564400115, 0.35866705527300125]
Estimation for 0.26 and 0.22 done
It took: 4.62369441986084 seconds.
[0.2685385781320009, 0.35777029564400115, 0.35866705527300125]
Estimation for 0.26 and 0.23 done
It took: 

Estimation for 0.28 and 0.17 done
It took: 4.523719072341919 seconds.
[0.25389933133400083, 0.3838601227310013, 0.34742298676500116]
Estimation for 0.28 and 0.18000000000000002 done
It took: 4.891347646713257 seconds.
[0.25949331284300087, 0.37286720470700124, 0.3529974695040012]
Estimation for 0.28 and 0.19 done
It took: 5.129128694534302 seconds.
[0.2658301686380009, 0.3595335776270012, 0.3602572156340012]
Estimation for 0.28 and 0.2 done
It took: 4.783859729766846 seconds.
[0.2705869087530009, 0.35037041267900115, 0.36473652200200124]
Estimation for 0.28 and 0.21000000000000002 done
It took: 4.817148208618164 seconds.
[0.2705869087530009, 0.35037041267900115, 0.36473652200200124]
Estimation for 0.28 and 0.22 done
It took: 5.4177234172821045 seconds.
[0.2705869087530009, 0.35037041267900115, 0.36473652200200124]
Estimation for 0.28 and 0.23 done
It took: 5.061611175537109 seconds.
[0.2705869087530009, 0.35037041267900115, 0.36473652200200124]
Estimation for 0.28 and 0.240000000000000

Estimation for 0.3 and 0.14 done
It took: 4.522279977798462 seconds.
[0.2400744858070008, 0.40459339947900136, 0.3414229491620011]
Estimation for 0.3 and 0.15000000000000002 done
It took: 5.0638368129730225 seconds.
[0.2438127458420008, 0.39670060323000134, 0.34559046033100116]
Estimation for 0.3 and 0.16 done
It took: 5.849185943603516 seconds.
[0.24937521392700082, 0.3874798187330013, 0.3490537601440012]
Estimation for 0.3 and 0.17 done
It took: 6.10212779045105 seconds.
[0.25702246596500083, 0.37404729128500125, 0.35481017811500115]
Estimation for 0.3 and 0.18000000000000002 done
It took: 4.679445743560791 seconds.
[0.26363944932600086, 0.36099966272700124, 0.36142304553400123]
Estimation for 0.3 and 0.19 done
It took: 5.4564008712768555 seconds.
[0.2695334162050009, 0.3480985990420012, 0.3687275851070012]
Estimation for 0.3 and 0.2 done
It took: 5.004678726196289 seconds.
[0.2730671673280009, 0.3412557914180011, 0.37211085777000125]
Estimation for 0.3 and 0.21000000000000002 done
I

In [48]:
# list_of_dataframes_with_results = []
# for component_number in range(len(spectra)):
#     results_for_component = pd.DataFrame(columns=np.arange(start=lower_limit, stop=upper_limit, step=step),
#                                    index=np.arange(start=lower_limit, stop=upper_limit, step=step))
#     for i in range(results_for_component.shape[0]):
#         for j in range(results_for_component.shape[1]):
#             results_for_component.iloc[i,j] = list_of_estimation_results[i][j][component_number]
    
#     list_of_dataframes_with_results.append(results_for_component)

In [49]:
# with open('results_for_different_kappas_variant_'+str(variant+1)+'.pkl', 'wb') as f:
#     pickle.dump(list_of_dataframes_with_results, f)

#### Molar proportions

In [50]:
with open('results_for_different_kappas_variant_'+str(variant+1)+'.pkl', 'rb') as f:
    list_of_dataframes_with_results = pickle.load(f)
#first coordinate: kappa,
#second coordinate: kappa_prime

In [51]:
temp = []
for nr_of_component, df in enumerate(list_of_dataframes_with_results):
    temp.append(df/protons[nr_of_component])
temp2 = []
for df in temp:
    temp2.append(df/sum(temp))
list_of_dataframes_with_results = temp2
del(temp)
del(temp2)

In [52]:
dataframes_ready_for_heatmap = []
for i, df in enumerate(list_of_dataframes_with_results):
    preprocessed_df = abs(df - molar_proportions[variant][i])
    dataframes_ready_for_heatmap.append(preprocessed_df)

In [53]:
all_components_results_molar = sum(dataframes_ready_for_heatmap)

#### Visible proportions

In [54]:
with open('results_for_different_kappas_variant_'+str(variant+1)+'.pkl', 'rb') as f:
    list_of_dataframes_with_results = pickle.load(f)
#first coordinate (rows): kappa,
#second coordinate (columns): kappa_prime

In [55]:
dataframes_ready_for_heatmap = []
for i, df in enumerate(list_of_dataframes_with_results):
    preprocessed_df = abs(df - real_visible_proportions[i])
    dataframes_ready_for_heatmap.append(preprocessed_df)

In [56]:
all_components_results_vis = sum(dataframes_ready_for_heatmap)

#### Both

In [57]:
all_components_results_both = all_components_results_vis + all_components_results_molar

In [58]:
all_components_results_both = all_components_results_both.apply(pd.to_numeric, errors = 'coerce', axis=0)

In [59]:
all_components_results_both.min().min()

0.034958769932540545

In [64]:
all_components_results_both[0.09][0.23]
all_components_results_both[0.060000000000000005][0.14]
all_components_results_both[0.05][0.09]
all_components_results_both[0.060000000000000005][0.12]

0.034958769932540545

### Saving preprocessed spectra

In [41]:
# for i, sp in enumerate(spectra_and_mixture):
#     try:
#         np.savetxt('preprocessed_variant_'+str(variant+1)+'_'+str(names[i])+'.csv', np.array(sp.confs), delimiter=',')
#     except IndexError:
#         np.savetxt('preprocessed_mix_variant_'+str(variant+1)+'.csv', np.array(sp.confs), delimiter=',')